In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from shapely.geometry import Point

# --- 1. Charger le NetCDF ---
ds = xr.open_dataset("era5_t2m_20250101_fr.nc")
var_name = "t2m"   # à adapter si besoin

print(ds)

# --- 2. Extraire les lat/lon ---
lats = ds["latitude"].values   # (n_lat,)
lons = ds["longitude"].values  # (n_lon,)

# --- 3. Charger la France (shapefile) ---
france = gpd.read_file("../daaata/region.shx")   # ton fichier
gdf = france.to_crs("EPSG:4326")                 # WGS84

# Union de tous les polygones France
france_poly = gdf.unary_union

# --- 4. Construire la grille de points centres des pixels ---
lon2d, lat2d = np.meshgrid(lons, lats)  # lon: axe x, lat: axe y

# Aplatir pour test point-in-polygon
points = [Point(x, y) for x, y in zip(lon2d.ravel(), lat2d.ravel())]

# ⚠️ Lent mais fait une fois, pas par heure
mask_flat = np.array([france_poly.contains(p) for p in points])
mask = mask_flat.reshape(lat2d.shape)  # (n_lat, n_lon)

# --- 5. Appliquer le masque à tout le champ t2m (toutes les heures) ---
t2m = ds[var_name]   # (valid_time, latitude, longitude)
t2m_fr = t2m.where(mask)  # NaN en dehors de la France